In [ ]:
import pandas as pd
df_tennis = pd.read_csv('PlayTennis.csv')
df_tennis

,PlayTennis,Outlook,Temperature,Humidity,Wind
0,No,Sunny,Hot,High,Weak
1,No,Sunny,Hot,High,Strong
2,Yes,Overcast,Hot,High,Weak
3,Yes,Rain,Mild,High,Weak
4,Yes,Rain,Cool,Normal,Weak
5,No,Rain,Cool,Normal,Strong
6,Yes,Overcast,Cool,Normal,Strong
7,No,Sunny,Mild,High,Weak
8,Yes,Sunny,Cool,Normal,Weak
9,Yes,Rain,Mild,Normal,Weak


In [ ]:
import math
def entropy(probs):
  return sum([-prob*math.log(prob,2) for prob in probs])

def entropy_of_list(ls):
  from collections import Counter
  class_counter = Counter(x for x in ls)
  total_instances = len(ls)
  probs = [x/total_instances for x in class_counter.values()]
  return entropy(probs)

In [ ]:
def information_gain(df,split_attr,tg_attr):
  df_split = df.groupby(split_attr)
  nobs = len(df.index)
  df_agg = df_split.agg({tg_attr:[entropy_of_list,lambda x : len(x)/nobs]})[tg_attr]
  df_agg.columns = ['Entropy','Proportion']

  old_entropy = entropy_of_list(df[tg_attr])
  new_entropy = sum(df_agg['Entropy']*df_agg['Proportion'])
  return old_entropy-new_entropy

In [ ]:
def id3(df,tg_attr,attr,default_class=None,default_attr='S'):
  from collections import Counter
  pos_neg = Counter(x for x in df[tg_attr])
  print(pos_neg,len(pos_neg))
  if len(pos_neg)==1:
    return(next(iter(pos_neg)))
  elif df.empty or (not attr):
    return default_class
  else:
    default_class = max(pos_neg)
    print(f'default_class: {default_class}')
    gains = {}
    print("\nOn attribute: ",default_attr,attr)
    for a in attr:
      gains[a] = information_gain(df,a,tg_attr)
      print(f'\nInformation gain on {a} is {gains[a]:0.4f}')
    best_attr = max(gains,key = lambda x: gains[x])
    print("\nThe attribute with maximum information gain is ",best_attr)
    tree = { best_attr:{}}
    attr.remove(best_attr)
    for att_val,data in df.groupby(best_attr):
      subtree = id3(data,tg_attr,attr,default_class,best_attr)
      tree[best_attr][att_val]=subtree
    print('best_attr,att_val',best_attr,att_val)
    print("\nTree",tree)
  return tree

       

In [ ]:
attr = list(df_tennis.columns)
print("The list of attributes are: ",attr)
attr.remove('PlayTennis')
print("The predicting attributes are: ",attr)

The list of attributes are:  ['PlayTennis', 'Outlook', 'Temperature', 'Humidity', 'Wind']
The predicting attributes are:  ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [ ]:
from pprint import pprint
tree = id3(df_tennis,'PlayTennis',attr)
print("\nThe Resultant tree \n")
pprint(tree)

Counter({'Yes': 9, 'No': 5}) 2
default_class: Yes

On attribute:  S ['Outlook', 'Temperature', 'Humidity', 'Wind']

Information gain on Outlook is 0.2467

Information gain on Temperature is 0.0292

Information gain on Humidity is 0.1518

Information gain on Wind is 0.0481

The attribute with maximum information gain is  Outlook
Counter({'Yes': 4}) 1
Counter({'Yes': 3, 'No': 2}) 2
default_class: Yes

On attribute:  Outlook ['Temperature', 'Humidity', 'Wind']

Information gain on Temperature is 0.0200

Information gain on Humidity is 0.0200

Information gain on Wind is 0.9710

The attribute with maximum information gain is  Wind
Counter({'No': 2}) 1
Counter({'Yes': 3}) 1
best_attr,att_val Wind Weak

Tree {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}
Counter({'No': 3, 'Yes': 2}) 2
default_class: Yes

On attribute:  Outlook ['Temperature', 'Humidity']

Information gain on Temperature is 0.5710

Information gain on Humidity is 0.9710

The attribute with maximum information gain is  Humidity
Cou